In [1]:
# ONTARIO
import requests 
import pandas as pd

URL = "https://www.ieso.ca/ieso/api/PowerData/MarketAtAGlance?"
r = requests.get(url=URL)
data = r.json()
supply_data = data['Subset'][2]

keys_to_ignore = ['Generator Availability', 'Generator Availability at Peak', 'Hourly Imports', 'Hourly Exports', 'Maximum Ontario Import Capacity']
supply_data['Information']
current_supply = {}
for info in supply_data['Information']:
  key = info['InfoTitle']['Value']
  if key not in keys_to_ignore:
    value = info['NewVal']
    if value == '':
      current_supply[key] = 0
    else: 
      current_supply[key] = int(value)

df = pd.DataFrame.from_dict(current_supply, orient='index', columns=['Output'])
df['Percentage'] = df['Output'] / df['Output'].sum()
df

,Output,Percentage
Nuclear,8469,0.487818
Coal,0,0.000000
Hydro,4065,0.234145
Gas,1226,0.070618
Biofuel,132,0.007603
Solar,8,0.000461
Wind,3461,0.199355


In [2]:
# ALBERTA
import requests 
import pandas as pd

URL = 'http://ets.aeso.ca/ets_web/ip/Market/Reports/CSDReportServlet'
r = requests.get(url=URL)

# NOTE OTHER IS CONSIDERED BIOFUEL
data = pd.read_html(r.text, match='GENERATION', header=0, skiprows=1)[0]
data.columns=['Group', 'Maximum Capacity', 'Total Net Generation', 'Dispatched Contingency Reserve']
data['Percentage'] = data['Total Net Generation'] / data['Total Net Generation'].iloc[:-1].sum()
data

,Group,Maximum Capacity,Total Net Generation,Dispatched Contingency Reserve,Percentage
0,GAS,9573,5762,43,0.601336
1,HYDRO,894,175,252,0.018263
2,ENERGY STORAGE,50,0,50,0.000000
3,SOLAR,336,1,0,0.000104
4,WIND,2139,1586,0,0.165519
5,OTHER,433,316,12,0.032979
6,DUAL FUEL,540,243,5,0.025360
7,COAL,2530,1499,0,0.156439
8,TOTAL,16495,9582,362,1.000000


In [3]:
# QUEBEC
import requests 
import pandas as pd

URL = "https://www.hydroquebec.com/data/documents-donnees/donnees-ouvertes/json/production.json"
# Other is considered biofuel https://www.hydroquebec.com/documents-data/open-data/electricity-generation-quebec/
translation = {
    'hydraulique': 'hydro',
    'eolien': 'wind',
    'autres': 'biofuel',
    'solaire': 'solar',
    'thermique': 'thermal',
    'total': 'total'
}
r = requests.get(url=URL)
data = r.json()
latest_info = {}
for energy_info in reversed(data['details']):
  if energy_info['valeurs']['total'] != 0.0:
    latest_info = energy_info
    break

latest_info_english = {}
for source_type in latest_info['valeurs']:
  latest_info_english[translation[source_type]] = latest_info['valeurs'][source_type]

df = pd.DataFrame.from_dict(latest_info_english, orient='index', columns=['Output'])

# Reverse sort the dataframe so total is last row
df = df.iloc[::-1]
df['Percentage'] = df['Output'] / df['Output'].iloc[:-1].sum()
df


,Output,Percentage
solar,0.0,0.000000
biofuel,563.0,0.022265
wind,799.0,0.031599
hydro,23924.0,0.946136
total,25286.0,1.000000
